#### Code to validate models

In [230]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [232]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from collections import Counter
from tqdm.notebook import tqdm

In [233]:
from params import *

In [234]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample

from utils.logger import Config

from utils.metrics import per_class_average_precision_score, study_level_map, study_level_map_binary

## Data

### Load

In [253]:
EXP_FOLDERS = [
    LOG_PATH + "2021-07-30/4/",
#     LOG_PATH + "2021-07-31/0/",
#     LOG_PATH + "2021-08-01/0/",
#     LOG_PATH + "2021-08-01/1/",
]

EXP_FOLDER = EXP_FOLDERS[-1]

USE_TTA = True

In [254]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [255]:
if USE_TTA:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img_flip.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study_flip.npy") for f in EXP_FOLDERS], 0)
else:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study.npy") for f in EXP_FOLDERS], 0)

In [267]:
df = pd.read_csv(EXP_FOLDER + "data.csv")

pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
df['pred_img'] = pred_oof_img

In [268]:
df = df[df['kfold'] == 2].reset_index(drop=True)

In [269]:
ap = per_class_average_precision_score(df['pred_img'].values, df['img_target'].values)
print(f'Image mAP : {ap :.4f}')

auc = roc_auc_score(df['img_target'], df['pred_img'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(df['img_target'], df['pred_img'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9632
Image AUC : 0.9173
Image Acc : 0.8491


In [270]:
study_level_map_binary(df['indeterminate_pred'], df['study_label'] == 'indeterminate', df['study_id'])

0.32469325308563324

In [260]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.4f}')

Study mAP : 0.3784


### Post-process

#### Img merging

In [175]:
groups = df[['study_id', 'pred_img']].groupby('study_id').mean().rename(
    columns={'pred_img': 'pred_img_merged'}
).reset_index()
df_ = df.merge(groups, on="study_id", how="left")

# df_.loc[df_['negative_pred'] > 0.75, 'pred_img'] = 0
# df_.loc[df_['typical_pred'] > 0.75, 'pred_img'] = 1
# df_.loc[df_['indeterminate_pred'] > 0.75, 'pred_img'] = 1

In [176]:
ap = per_class_average_precision_score(df_['pred_img_merged'].values, df['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(df_['img_target'], df_['pred_img_merged'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(df_['img_target'], df_['pred_img_merged'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9605
Image AUC : 0.9110
Image Acc : 0.8470


#### Study using img

In [177]:
df_ = df.copy()

p = 0.5
df_['negative_pred'] *= (1 - df_['pred_img']) ** p
df_['typical_pred'] *= (df_['pred_img']) ** p

# df_['indeterminate_pred'] *= (df_['pred_img']) ** p
# df_['atypical_pred'] *= (df_['pred_img']) ** p

study_map = study_level_map(df_[pred_cols].values, df_[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.5f}')

Study mAP : 0.38327


In [178]:
df_study = df[
    ['study_id'] + pred_cols + CLASSES + ['img_target', 'pred_img']
].groupby('study_id').agg(np.mean).copy()

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study['typical_pred'] *= df_study['pred_img'] 
# df_study['indeterminate_pred'] *= df_study['pred_img'] 
# df_study['atypical_pred'] *= df_study['pred_img'] 

df_study.loc[df_study['pred_img'] > 0.75, 'negative_pred'] *= 0.5
df_study.loc[df_study['pred_img'] < 0.2, 'negative_pred'] *= 2

df_study.loc[df_study['pred_img'] > 0.75, 'typical_pred'] *= 1.2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 1.1

df_study.loc[df_study['pred_img'] < 0.25, 'typical_pred'] *= 0.8
# df_study.loc[df_study['pred_img'] < 0.2, 'indeterminate_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'atypical_pred'] *= 0.9

accs = per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)
np.round(accs, 4), np.round(np.mean(accs) * 2/3, 4)

(array([0.802 , 0.8478, 0.3069, 0.3421]), 0.3831)

## Results

In [179]:
def proc(x):
    x = x.split('0 0 1 1')[:4]
    x = [float(y.strip().split(' ')[1]) for y in x]
    return x

In [180]:
sub = pd.read_csv("../output/sub_0931.csv")

In [181]:
sub = pd.read_csv("../output/sub_0931.csv")
sub_study = sub[sub['id'].apply(lambda x: "study" in x)].copy()

pred_test = np.array(sub_study['PredictionString'].apply(proc).values.tolist())

for i, c in enumerate(CLASSES):
    sub_study[c] = pred_test[:, i]
    
sub_study.to_csv("../output/sub_0931_study.csv", index=False)

In [182]:
sub = pd.read_csv("../output/sub_0931.csv")

sub_img = sub[sub['id'].apply(lambda x: "study" not in x)].copy()
sub_img['none'] = sub_img['PredictionString'].apply(lambda x: float(x.split('none')[-1].strip().split(' ')[0]))

sub_img.to_csv("../output/sub_0931_img.csv", index=False)

In [183]:
def treat_oof(pred_oof_study):
    pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])
    
    df_study = pd.read_csv(DATA_PATH + "train_study_level.csv")
    df_study['study_id'] = df_study['id'].apply(lambda x: x.split('_')[0])
    df_study = df_study.rename(columns={c: c.split(' ')[0].lower() for c in df_study.columns})

    df_study.drop('id', axis=1, inplace=True)

    df_study = df_study.merge(pred_oof_study, how="left", left_on="study_id", right_on="id").dropna()
    
    pred_oof_study = np.array(df_study['PredictionString'].apply(proc).values.tolist())
    
    df_study[pred_cols] = pred_oof_study
    
    df_g = df[['study_id'] + pred_cols].groupby('study_id').mean().reset_index()
    df_study = df_study.merge(df_g, how="left", left_on="id", right_on="study_id", suffixes=['', '_theo'])
    
    return df_study

In [184]:
df_old = treat_oof(pd.read_csv('../output/OOF_study_only_EBV2M_768.csv'))
df_v2m_2 = treat_oof(pd.read_csv('../output/oof_v2m.csv'))
df_b4 = treat_oof(pd.read_csv('../output/oof_b4.csv'))
df_b5 = treat_oof(pd.read_csv('../output/oof_b5.csv'))
df_ono = treat_oof(pd.read_csv('../output/oof_ono.csv')[['id', 'PredictionString']])

In [185]:
df_theo = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').mean().reset_index()
df_theo = df_v2m_2[['id']].merge(df_theo, how="left", left_on="id", right_on="study_id")

df_ono = df_v2m_2[['id']].merge(df_ono, how="left", left_on="id", right_on="study_id")

In [186]:
p = 1
oofs = [
#     df_old[pred_cols].values ** p * 1,
    df_theo[pred_cols].values ** p * 1,
    df_v2m_2[pred_cols].values ** p * 1,
#     df_b4[pred_cols].values ** p * 0.1,
    df_b5[pred_cols].values ** p * 0.5,
#     df_ono[pred_cols].values ** p * 0.1,
]

oof = np.mean(oofs, 0)

In [187]:
oof.max(0)

array([0.81562984, 0.81986495, 0.45891351, 0.71444997])

In [188]:
CLASSES

['negative', 'typical', 'indeterminate', 'atypical']

In [189]:
from scipy.stats import pearsonr

In [190]:
# truth = df_theo[CLASSES].values
# pred = oof
# aucs = [roc_auc_score(truth[:, i].flatten(), pred[:, i].flatten()) for i in range(pred.shape[1])]
# aucs

In [191]:
# for i in range(4):
#     print(pearsonr(pred[:, 2], pred[:, i])[0])

In [192]:
per_class_average_precision_score(oof, df_theo[CLASSES].values, num_classes=NUM_CLASSES) * 2 / 3

0.39189165578531077

In [193]:
per_class_average_precision_score(oof, df_theo[CLASSES].values, num_classes=NUM_CLASSES, average=False) #* 2 / 3

array([0.81115651, 0.85174298, 0.3206541 , 0.36779635])

## Blend img

In [216]:
oof_mk = pd.read_csv("../output/oof_image_level_mk.csv")

In [218]:
roc_auc_score(oof_mk['target'], oof_mk['pred'])

0.8851199771832216

In [195]:
oof_mk = oof_mk.merge(df, on='image_id', how='left')[['image_id', 'img_target', 'pred_img', 'pred']].dropna()

In [196]:
ap = per_class_average_precision_score(oof_mk['pred'].values, oof_mk['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(oof_mk['img_target'], oof_mk['pred'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(oof_mk['pred'] > 0.5, oof_mk['img_target'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9528
Image AUC : 0.8916
Image Acc : 0.8339


In [197]:
ap = per_class_average_precision_score(oof_mk['pred_img'].values, oof_mk['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(oof_mk['img_target'], oof_mk['pred_img'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(oof_mk['pred_img'] > 0.5, oof_mk['img_target'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9604
Image AUC : 0.9109
Image Acc : 0.8460


In [198]:
oof_mk['blend'] = (oof_mk['pred_img'] * 4 + oof_mk['pred']) / 5

In [199]:
ap = per_class_average_precision_score(oof_mk['blend'].values, oof_mk['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(oof_mk['img_target'], oof_mk['blend'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(oof_mk['blend'] > 0.5, oof_mk['img_target'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9615
Image AUC : 0.9123
Image Acc : 0.8487
